### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

In [243]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [244]:
# Create list of unique Screen Names to find total players
players = purchase_data["SN"].unique()
playerCount_df = pd.DataFrame({"Total Players":[len(players)]})
playerCount_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [245]:
# Create list of unique Item IDs to find total items
uniqueItem = purchase_data["Item ID"].unique()

# Calculate average price, total purchases, and total revenue
averagePrice = purchase_data["Price"].mean()
NumberOfPurchases = purchase_data["Purchase ID"].count()
TotalRevenue = purchase_data["Price"].sum()

# Create dataframe with identified values
summary_df = pd.DataFrame({
    "Number of Unique Items":[len(uniqueItem)], 
    "Average Price":[averagePrice], 
    "Number of Purchases":[NumberOfPurchases], 
    "Total Revenue":[TotalRevenue]})

# Format numbers and print dataframe
summary_df["Average Price"] = summary_df["Average Price"].map('${:,.2f}'.format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].map('${:,.2f}'.format)
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [246]:
# Isolate Screen Names and Gender into a new dataframe to analyze
unique_playersG_df = purchase_data[["SN", "Gender"]]

# Since only looking at total amounts of unique players to analyze gender counts, can remove duplicated screen names
unique_playersG_df = unique_playersG_df.drop_duplicates('SN')

In [247]:
# Find gender value counts of unique Screen Names
unique_playersG_df["Gender"].value_counts()

Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64

In [248]:
# Find gender value counts of unique Screen Names
numbersByGender = unique_playersG_df["Gender"].value_counts()

# Translate above information into dataframe and rename column
genderNumbers = pd.DataFrame(numbersByGender)
genderNumbers = genderNumbers.rename(columns = {"Gender":"Total Count"})
genderNumbers.index = genderNumbers.index.rename("Gender")

# Calcuate total amount
total = genderNumbers.sum()

# Find percentages,add to a new column, format and display
genderNumbers['Percentage of Players'] = (genderNumbers/ total) * 100
genderNumbers['Percentage of Players'] = genderNumbers['Percentage of Players'].map('{0:.2f}%'.format)

genderNumbers

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [205]:
# Group by gender for all columns
GenderGroup = purchase_data.groupby(['Gender'])

# Create dataframe by calculating purchase count, average purchase, and total purchase
GenderGroup = GenderGroup.agg({"SN":['count'], "Price": ['mean', 'sum']})

# Remove multiIndex and rename columns
GenderGroup.columns = GenderGroup.columns.droplevel(0)
GenderGroup = GenderGroup.rename(columns = {
    "count":"Purchase Count",
    "mean":"Average Purchase Price",
    "sum":"Total Purchase Value"
})
GenderGroup

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,3.203009,361.94
Male,652,3.017853,1967.64
Other / Non-Disclosed,15,3.346000,50.19


In [206]:
# Merge the two dataframes into one along the index
GenderAnalytics_df = pd.merge(genderNumbers, GenderGroup, how='outer', on='Gender')

In [207]:
# Calculate the average purchase by player and add to dataframe as a new column
GenderAnalytics_df["Avg Total Purchase per Player"] = GenderAnalytics_df["Total Purchase Value"]/GenderAnalytics_df["Total Count"]

# Format column data and print
GenderAnalytics_df["Avg Total Purchase per Player"] = GenderAnalytics_df["Avg Total Purchase per Player"].map('${:,.2f}'.format)
GenderAnalytics_df["Average Purchase Price"] = GenderAnalytics_df["Average Purchase Price"].map('${:,.2f}'.format)
GenderAnalytics_df["Total Purchase Value"] = GenderAnalytics_df["Total Purchase Value"].map('${:,.2f}'.format)
GenderAnalytics_df

,Total Count,Percentage of Players,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Player
Gender,,,,,,
Male,484,84.03%,652,$3.02,"$1,967.64",$4.07
Female,81,14.06%,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,11,1.91%,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [171]:
# Isolate Screen Name and Ages into a new dataframe
unique_playersA_df = purchase_data[["SN", "Age"]]

# Since only calculating the total number of unique players per age, can drop duplicates of screen name 
unique_playersA_df = unique_playersA_df.drop_duplicates('SN')

In [172]:
# Create slices for age groups
bins = [0, 9, 14, 19, 24, 29, 34, 39 , 100]
labels = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]

unique_playersA_df['Age Group'] = pd.cut(unique_playersA_df["Age"], bins = bins, labels = labels)

In [173]:
# Group data by age groups to find totals
ageGroup = unique_playersA_df.groupby(['Age Group'])
age_data = ageGroup["SN"].count()

# Create dataframe with total counts per age group and rename column
AgeData_df = pd.DataFrame(age_data)
AgeData_df = AgeData_df.rename(columns = {"SN":"Total Count"})
AgeData_df

,Total Count
Age Group,
<10,17
10-14,22
15-19,107
20-24,258
25-29,77
30-34,52
35-39,31
40+,12


In [174]:
# Calculate total players overall
total = age_data.sum()
total

# Calculate percentages of each age group on total players and add into new column with formatting and print
AgeData_df["Percentage of Players"] = (AgeData_df["Total Count"]/total)*100
AgeData_df["Percentage of Players"] = AgeData_df["Percentage of Players"].map('{0:.2f}%'.format)
AgeData_df

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [202]:
# Cut slices for age groups on main dataframe copy
purchaseByAge_df = purchase_data.copy()
purchaseByAge_df["Age Group"] = pd.cut(purchaseByAge_df["Age"], bins = bins, labels = labels)

# Reset index as age groups
purchaseByAge_df = purchaseByAge_df.set_index('Age Group')

# Group by the age group slices
purchaseByAge_df = purchaseByAge_df.groupby(by=['Age Group'])

In [203]:
# Calculate and create dataframe for purchase count, total purchase price, and average purchase price
purchaseAnalytics = purchaseByAge_df.agg({"Purchase ID": ["count"], "Price":["mean","sum"]})

# Display to verify information
purchaseAnalytics

Purchase ID     Price         
                count      mean      sum
Age Group                               
<10                23  3.353478    77.13
10-14              28  2.956429    82.78
15-19             136  3.035956   412.89
20-24             365  3.052219  1114.06
25-29             101  2.900990   293.00
30-34              73  2.931507   214.00
35-39              41  3.601707   147.67
40+                13  2.941538    38.24

In [204]:
# Remove multiIndex and rename columns
purchaseAnalytics.columns = purchaseAnalytics.columns.droplevel(0)
purchaseAnalytics = purchaseAnalytics.rename(columns = {
    "count" : "Purchase Count", 
    "mean": "Average Purchase Price", 
    "sum" : "Total Purchase Price" 
})

# Calculate average purchase per player and add into new column
purchaseAnalytics["Avg Total Purchase per Player"] = purchaseAnalytics["Total Purchase Price"]/AgeData_df["Total Count"]

# Format column data
purchaseAnalytics["Average Purchase Price"] = purchaseAnalytics["Average Purchase Price"].map('${:,.2f}'.format)
purchaseAnalytics["Total Purchase Price"] = purchaseAnalytics["Total Purchase Price"].map('${:,.2f}'.format)
purchaseAnalytics["Avg Total Purchase per Player"] = purchaseAnalytics["Avg Total Purchase per Player"].map('${:,.2f}'.format)

# Remove index column heading
purchaseAnalytics.index.name = None

# Print
purchaseAnalytics

,Purchase Count,Average Purchase Price,Total Purchase Price,Avg Total Purchase per Player
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [193]:
# Group data by screen names
SN_data = purchase_data.groupby(["SN"])

In [194]:
# Calculate and create dataframe with Purchase Coount, average purchase price, and total purchase amount for each player
playerSpending_df = SN_data.agg({"Purchase ID": ['count'], "Price":['mean', 'sum']})

# Remove multiIndex and rename columns
playerSpending_df.columns = playerSpending_df.columns.droplevel(0)
playerSpending_df = playerSpending_df.rename(columns = {
    "count":"Purchase Count",
    "mean":"Average Purchase Price",
    "sum":"Total Purchase Value"
})

# Sort descending by greatest purchase total
playerSpending_df = playerSpending_df.sort_values(by = ['Total Purchase Value'], ascending = False)

# Format column data and display preview
playerSpending_df["Average Purchase Price"] = playerSpending_df["Average Purchase Price"].map('${:,.2f}'.format) 
playerSpending_df["Total Purchase Value"] = playerSpending_df["Total Purchase Value"].map('${:,.2f}'.format)
playerSpending_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [237]:
# Isolate Item ID, Item Name and Price into new dataframe
popular_df = purchase_data[["Item ID","Item Name","Price"]]

# Group by Item ID and Item Name
popular_df = popular_df.groupby(['Item ID','Item Name'])

# Create dataframe from calculated values of grouped object
popItems_df = popular_df.agg({"Item ID": ["count"], "Price":["min","sum"]})

In [238]:
# Remove multiIndex, rename columns, and format data
popItems_df.columns = popItems_df.columns.droplevel(0)
popItems_df = popItems_df.rename(columns = {
    "count":"Purchase Count",
    "min":"Item Price",
    "sum":"Total Purchase Value"
})

# Create a copy of the dataframe to format column data - format mapping changes data type from float
# to an object; will need the numeric value in th next step
popItemCount_df = popItems_df.copy()
popItemCount_df["Item Price"] = popItemCount_df["Item Price"].map('${:,.2f}'.format)
popItemCount_df["Total Purchase Value"] = popItemCount_df["Total Purchase Value"].map('${:,.2f}'.format)

# Sort dataframe descending from greatest purchase count
popItemCount_df = popItemCount_df.sort_values(by = ['Purchase Count'], ascending = False)
popItemCount_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [241]:
# Sort dataframe descending by greatest total purchase value, format, and display preview 
popItems_df = popItems_df.sort_values(by = ['Total Purchase Value'], ascending = False)
popItems_df["Item Price"] = popItems_df["Item Price"].map('${:,.2f}'.format)
popItems_df["Total Purchase Value"] = popItems_df["Total Purchase Value"].map('${:,.2f}'.format)
popItems_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
